In [1]:
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
mask = np.load(r"C:\Users\DU\aman_fastmri\Data\mask_4x_320_random.npy")  # Shape: (1, 320, 320)
print("og shape:", mask.shape)

# # Use np.tile to reshape it to (1, 320, 320, 1)
# # var_sampling_mask = np.tile(var_sampling_mask[..., np.newaxis], (1, 1, 1, 1))  # Final shape: (1, 320, 320, 1)
# mask = np.tile(mask, (1, 320, 1, 2))  # tile height=320 times

# # Confirm final shape
# print("New shape:", mask.shape) 
# mask_for_plot = np.squeeze(mask[...,0])  # Shape: (320, 320)

# # Plot
# plt.figure(figsize=(5, 5))
# plt.imshow(mask_for_plot, cmap='gray')
# plt.title("Tiled Sampling Mask (320x320)")
# plt.axis('off')
# plt.show()

og shape: (1, 1, 320, 1)


In [3]:
import h5py
import numpy as np
import tensorflow as tf

def to_complex(x):
    return x[..., 0] + 1j * x[..., 1]

class MRISliceGeneratorMag(tf.keras.utils.Sequence):
    """
    Data generator for magnitude-only MRI reconstruction.

    Input  : undersampled magnitude image  (B, H, W, 1)
    Target : fully-sampled magnitude image (B, H, W, 1)
    """

    def __init__(self, file_list, batch_size=4, shuffle=True):
        self.file_list = file_list
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.slice_index_map = []
        self._build_index()

    def _build_index(self):
        for file_idx, file_path in enumerate(self.file_list):
            with h5py.File(file_path, 'r') as f:
                num_slices = f['image_under'].shape[0]
                for slice_idx in range(num_slices):
                    self.slice_index_map.append((file_idx, slice_idx))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.slice_index_map) / self.batch_size))

    def __getitem__(self, index):
        batch_map = self.slice_index_map[
            index * self.batch_size : (index + 1) * self.batch_size
        ]

        input_mag_batch = []
        target_mag_batch = []

        for file_idx, slice_idx in batch_map:
            with h5py.File(self.file_list[file_idx], 'r') as f:
                img_under = f['image_under'][slice_idx]  # (H, W, 2)
                img_full  = f['image_full'][slice_idx]   # (H, W, 2)

                # Convert to complex
                img_under_c = to_complex(img_under)
                img_full_c  = to_complex(img_full)

                # Magnitude
                img_under_mag = np.abs(img_under_c)
                img_full_mag  = np.abs(img_full_c)

                input_mag_batch.append(img_under_mag)
                target_mag_batch.append(img_full_mag)

        # Stack and add channel dimension
        x_batch = np.stack(input_mag_batch, axis=0)[..., np.newaxis]  # (B, H, W, 1)
        y_batch = np.stack(target_mag_batch, axis=0)[..., np.newaxis] # (B, H, W, 1)

        return x_batch, y_batch

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.slice_index_map)


In [4]:
val_folder = r"D:\fastmri_singlecoil_FSSCAN\val_norm"

In [5]:
import h5py
import numpy as np
import glob
import os

kspace_files_list_val = sorted(glob.glob(os.path.join(val_folder, "*.h5")))


half_val = len(kspace_files_list_val) 

kspace_files_list_val = kspace_files_list_val[:half_val]


val_gen = MRISliceGeneratorMag(kspace_files_list_val, batch_size=4, shuffle=False)


print(len(val_gen))  


1784


In [6]:
%run ./DCRCNN.ipynb


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# ============================================================
# Directory Setup
# ============================================================
save_dir = "./SavedModels_DCRCNN_full"
H, W = 320, 320
EPOCHS = 50
LEARNING_RATE = 1e-4

In [8]:
model = build_dcr_cnn(
    input_shape=(320, 320, 1),
    num_dcr_blocks=10,   # or 3 / 8
    num_features=64,
    growth_rate=32
)


# ============================================================
# Optimizer & Compile
# ============================================================
optimizer = Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss="mse")

# ============================================================
# Load Initial Weights (Optional Resume)
# ============================================================
if tf.train.latest_checkpoint(save_dir):
    model.load_weights(tf.train.latest_checkpoint(save_dir))
    print("✅ Loaded latest checkpoint")
else:
    print("ℹ️ No checkpoint found. Training from scratch.")

✅ Loaded latest checkpoint


In [14]:
import os
import numpy as np
import h5py
import glob
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# # Path to validation folder
# # val_folder = "F:/denoised_preprocessed_h5_val"

# val_folder = r"E:\fastmri\val_norm"
# # val_folder = r"D:\val_norm"

# # val_folder = r"G:\val_norm\val_norm"
# # files = sorted([os.path.join(val_folder, f) for f in os.listdir(val_folder) if f.endswith(".h5")])
# kspace_files_list_val = sorted(glob.glob(os.path.join(val_folder, "*.h5")))
file_paths = kspace_files_list_val


# ----------------------
# HELPERS
# ----------------------
def to_complex(x):
    return x[..., 0] + 1j * x[..., 1]

def nmse(gt, pred):
    return np.linalg.norm(gt - pred) ** 2 / (np.linalg.norm(gt) ** 2 + 1e-10)

def compute_ssim(gt, pred, max_val):
    return structural_similarity(
        gt, pred,
        data_range=max_val,
        win_size=9,
        gaussian_weights=False,
        use_sample_covariance=False,
        K1=0.01,
        K2=0.03
    )

# ----------------------
# STORAGE
# ----------------------
ssim_list = []
psnr_list = []
nmse_list = []

# ----------------------
# PROCESSING
# ----------------------
for file in tqdm(kspace_files_list_val, desc="Processing volumes"):
    with h5py.File(file, 'r') as f:
        image_under = f["image_under"][:]   # (S, H, W, 2)
        image_full   = f["image_full"][:]     # (S, H, W, 2)
        max_val = float(f["max_val_full_image"][0])

    num_slices = image_under.shape[0]
        # Convert to complex
    img_under_c = to_complex(image_under)
    img_full_c  = to_complex(image_full)

    # Magnitude
    img_under_mag = np.abs(img_under_c)
    img_full_mag  = np.abs(img_full_c)
    img_full_mag = np.expand_dims(img_full_mag, axis=-1)


    # --------------------------------------------------
    # TILE MASK FOR THIS VOLUME
    # --------------------------------------------------
    #mask_batch = np.tile(mask, (num_slices, 1, 1, 1))

    # --------------------------------------------------
    # MODEL INFERENCE
    # --------------------------------------------------
    pred = model.predict(
        img_under_mag,
        batch_size=1,
        verbose=0
    )
    #print("pred",pred.shape)
    #print("img_full_mag",img_full_mag.shape)
    

    max_val=img_full_mag.max()
    # image_full *= max_val
    
    # pred *= max_val  # Scale predicted output to original intensity range
    # #psnr_val = peak_signal_noise_ratio(image_full, pred, data_range=max_val)

    # # Convert to complex and get magnitude
    # gt_mag = np.abs(to_complex(image_full))
    # pred_mag = np.abs(to_complex(pred))

    # Volume-wise PSNR and NMSE
    psnr_val = peak_signal_noise_ratio(img_full_mag, pred, data_range=max_val)
    nmse_val = nmse(img_full_mag.flatten(), pred.flatten())

    psnr_list.append(psnr_val)
    nmse_list.append(nmse_val)

    # Slice-wise SSIM
    for i in range(img_full_mag.shape[0]):
        ssim_val = compute_ssim(
            img_full_mag[i, ..., 0],   # (320, 320)
            pred[i, ..., 0],           # (320, 320)
            max_val
        )
        ssim_list.append(ssim_val)
# ----------------------
# REPORT
# ----------------------
print("\n" + "=" * 40)
print(f"PSNR : {np.mean(psnr_list):.4f} ± {np.std(psnr_list):.4f} dB")
print(f"NMSE (Mag, volume): {np.mean(nmse_list):.6f} ± {np.std(nmse_list):.6f}")
print(f"SSIM (Mag, slice):  {np.mean(ssim_list):.4f} ± {np.std(ssim_list):.4f}")

print("=" * 40)


Processing volumes: 100%|████████████████████████████████████████████████████████████| 199/199 [04:09<00:00,  1.26s/it]


PSNR : 34.1764 ± 2.3373 dB
NMSE (Mag, volume): 0.017163 ± 0.006873
SSIM (Mag, slice):  0.8336 ± 0.0758
